"""
A. Tham khảo hưỡng dẫn về GNN tại:
+ https://viblo.asia/p/gioi-thieu-ve-graph-neural-networks-gnns-yZjJYG7MVOE
+ https://docs.dgl.ai/en/0.8.x/tutorials/blitz/4_link_predict.html
+ https://arxiv.org/ftp/arxiv/papers/1812/1812.08434.pdf


B. Yêu cầu:
1. Tìm hiểu và trình bày tổng quan về GNN
2. Sử dụng GCN để dự đoán liên kết mạng xã hội mầ bạn lựa chọn
+ https://stellargraph.readthedocs.io/en/stable/demos/link-prediction/gcn-link-prediction.html
3. Đánh giá kết quả giữa GCN và các phương pháp trong LAB 04.01 / 04.02
"""

Graph Neural Network (GNN) là một loại mạng nơ-ron được thiết kế để hoạt động trên dữ liệu dạng đồ thị. Dữ liệu này bao gồm tập hợp các đỉnh (nodes) và cạnh (edges), thường được biểu diễn dưới dạng 𝐺=(𝑉,𝐸)  
Trong đó:  
V: Tập hợp các đỉnh (nodes) đại diện cho thực thể.  
E: Tập hợp các cạnh (edges) biểu diễn mối quan hệ giữa các đỉnh.  
GNN tận dụng cấu trúc của đồ thị để học các đặc trưng (features) của đỉnh, cạnh, hoặc toàn bộ đồ thị, giúp giải quyết các bài toán phức tạp liên quan đến mạng lưới hoặc mối quan hệ.

# **Lý thuyết, Công thức, Ứng dụng của Graph Neural Network (GNN)**

## **1. Lý thuyết cơ bản**
Một đồ thị được biểu diễn như sau:
$$
G = (V, E)
$$
Trong đó:
- V: Tập các đỉnh (nodes).
- E: Tập các cạnh (edges).

Mục tiêu của GNN là học biểu diễn (embedding) từ đồ thị để giải quyết các bài toán như phân loại, dự đoán liên kết, hoặc học tập toàn đồ thị.

---

## **2. Công thức chính**

### **2.1. Biểu diễn cơ bản của GNN**
Quá trình cập nhật đặc trưng tại một đỉnh v sau k-bước:  
$$
h_v^{(k)} = f \left( h_v^{(k-1)}, \text{AGG} \left( \{ h_u^{(k-1)} : u \in \mathcal{N}(v) \} \right) \right)
$$  
Trong đó:  
-  h_v^{(k)} : Biểu diễn (embedding) của đỉnh \( v \) tại bước \( k \).
- \mathcal{N}(v) $$: Hàng xóm của đỉnh \( v \).
- \text{AGG} $$: Hàm tổng hợp (aggregation function), ví dụ: tổng (\( \sum \)), trung bình (\( \text{mean} \)), hoặc tối đa (\( \max \)).
- f: Hàm phi tuyến (thường là mạng nơ-ron hoặc ReLU).

---

### **2.2. Hàm mất mát**
Đối với bài toán phân loại đỉnh:  
$$
\mathcal{L} = \sum_{v \in V_{\text{train}}} \text{CrossEntropy}(y_v, \hat{y}_v)
$$  
Trong đó:  
- y_v: Nhãn thực tế của đỉnh v .
- : Giá trị dự đoán của mô hình tại v.

Đối với bài toán dự đoán liên kết:  
$$
\mathcal{L} = - \sum_{(u, v) \in E_{\text{train}}} \left( y_{uv} \log \hat{y}_{uv} + (1 - y_{uv}) \log (1 - \hat{y}_{uv}) \right)
$$  
Trong đó:
- y_{uv}: Nhãn thực tế cho liên kết giữa u và v.
- \hat{y}_{uv}: Dự đoán cho liên kết giữa u và v.

---

### **2.3. Học tập toàn đồ thị**
Học biểu diễn toàn đồ thị được định nghĩa như sau:  
$$
h_G = \text{READOUT} \left( \{ h_v : v \in V \} \right)
$$
Trong đó:  
- READOUT: Hàm tóm tắt
- h_G: Biểu diễn của toàn bộ đồ thị.

---

## **3. Ứng dụng**

### **3.1. Phân loại đỉnh (Node Classification)**
Dự đoán thuộc tính của từng đỉnh, ví dụ: phân loại người dùng trong mạng xã hội.

### **3.2. Dự đoán liên kết (Link Prediction)**
Dự đoán mối quan hệ giữa hai đỉnh, ví dụ: gợi ý kết bạn, dự đoán tương tác thuốc.

### **3.3. Phân loại đồ thị (Graph Classification)**
Phân loại toàn bộ đồ thị, ví dụ: dự đoán tính chất hóa học của phân tử.

### **3.4. Hệ thống gợi ý (Recommendation System)**
Sử dụng GNN để cải thiện hệ thống gợi ý dựa trên mạng lưới người dùng-sản phẩm.

---

## **4. Các biến thể phổ biến**
- **GCN (Graph Convolutional Network)**: Sử dụng phép tích chập trên đồ thị.
- **GAT (Graph Attention Network)**: Sử dụng attention để gán trọng số khác nhau cho các hàng xóm.
- **GraphSAGE**: Lấy mẫu (sampling) và tổng hợp (aggregation) trên hàng xóm.
- **GIN (Graph Isomorphism Network)**: Tối ưu hóa cho việc phân loại đồ thị.
- **RGNN (Relational GNN)**: Phù hợp với đồ thị không đồng nhất (heterogeneous graph).


In [23]:
import numpy as np
import pandas as pd
import networkx as nx
import stellargraph as sg
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding
from stellargraph.data import EdgeSplitter
from tensorflow.keras import layers, optimizers, losses, Model
from sklearn import metrics


In [45]:
from stellargraph.datasets import Cora

# Tải bộ dữ liệu Cora
dataset = Cora()
G, node_subjects = dataset.load()


In [46]:
# Thêm thuộc tính nút giả (node features)
for node in graph.nodes():
    graph.nodes[node]["feature"] = np.random.rand(10)  # 10 đặc trưng ngẫu nhiên

# Chuyển đổi sang StellarGraph
G = sg.StellarGraph.from_networkx(graph, node_features="feature")

# Tách dữ liệu thành tập train/test
edge_splitter = EdgeSplitter(G)
G_train, edges_train, labels_train = edge_splitter.train_test_split(
    p=0.1, method="global"
)

** Sampled 29 positive and 29 negative edges. **


In [47]:
#Xây dựng trình tạo dữ liệu (data generator)
generator = FullBatchLinkGenerator(G_train, method="gcn")
train_gen = generator.flow(edges_train, labels_train)

Using GCN (local pooling) filters...


In [52]:
# Xây dựng mô hình GCN
from tensorflow.keras.regularizers import l2

gcn = GCN(
    layer_sizes=[16, 16],
    activations=["relu", "relu"],
    generator=generator,
    dropout=0.5,
    kernel_regularizer=l2(0.01),
)

x_inp, x_out = gcn.in_out_tensors()

# Thêm tầng LinkEmbedding để dự đoán liên kết
link_prediction = LinkEmbedding(activation="sigmoid", method="ip")(x_out)
model = Model(inputs=x_inp, outputs=link_prediction)


In [53]:
# Compile mô hình
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.binary_crossentropy,
    metrics=["accuracy"],
)

# Huấn luyện mô hình
history = model.fit(
    train_gen, epochs=100, verbose=1, shuffle=False, validation_data=None
)

Epoch 1/100
1/1 [==============================] - 1s 790ms/step - loss: 1.0010 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 0.9773 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 19ms/step - loss: 0.9394 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 17ms/step - loss: 0.9274 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 17ms/step - loss: 0.9124 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8952 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8886 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 0.8788 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 0.8692 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8570 - accuracy: 0.5172
Epoch 11/100
1/1 [

In [54]:
# Dự đoán trên tập test
test_gen = generator.flow(edges_test, labels_test)
test_predictions = model.predict(test_gen)

# Đảm bảo test_predictions là mảng 1D
test_predictions = test_predictions.squeeze()  # Loại bỏ các chiều không cần thiết

# Đánh giá mô hình
roc_auc = metrics.roc_auc_score(labels_test, test_predictions)
print(f"Test ROC AUC: {roc_auc:.4f}")

# In báo cáo chính xác
print(metrics.classification_report(labels_test, np.round(test_predictions)))

1/1 [==============================] - 0s 103ms/step
Test ROC AUC: 0.5592
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       0.50      1.00      0.67        26

    accuracy                           0.50        52
   macro avg       0.25      0.50      0.33        52
weighted avg       0.25      0.50      0.33        52



c:\Users\DELL\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DELL\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DELL\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# So sánh GCN với các phương pháp khác
**Bảng đánh giá các phương pháp cổ điển:**

+ Các phương pháp như Common Neighbors, Jaccard Coefficient, và Adamic-Adar đạt được kết quả tốt trong việc dự đoán liên kết trên đồ thị đơn giản.
+ Common Neighbors có Precision cao nhất nhưng Recall thấp.
+ Adamic-Adar có sự cân bằng hơn giữa các chỉ số Accuracy, Precision, và Recall.  

**Kết quả GCN:**

ROC AUC của GCN chỉ đạt 0.5592, thấp hơn đáng kể so với các phương pháp truyền thống (ví dụ: Common Neighbors đạt 0.9474).  
Precision cho class 0 (negative edges) = 0, điều này cho thấy mô hình không dự đoán đúng bất kỳ cạnh âm nào.  
Recall cho class 1 (positive edges) = 1, nghĩa là mô hình chỉ dự đoán đúng các cạnh dương, dẫn đến vấn đề mất cân bằng.  